> Importing all necessary Files

In [1]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
import pandas as pd
import pickle

import tensorflow as tf
import os
import numpy as np
import pandas as pd
from tensorflow import keras

# import tensorflow as tf
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.models import Model



#importing tensorflow
from tensorflow.keras.layers import Dense,Input,Activation,Dropout,Flatten,MaxPool1D,concatenate,Embedding,BatchNormalization
from keras.layers import CuDNNLSTM,SpatialDropout1D,GlobalMaxPooling1D,GlobalAveragePooling1D,Concatenate,Add,CuDNNGRU
from tensorflow.keras.models import Model,Sequential
import random as rn


import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random

import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
import pandas as pd
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
pd.options.mode.chained_assignment = None

In [3]:
# Data = pd.read_csv(r"C:\Users\Lenovo\Downloads\Self case study 2\Data\jigsaw-unintended-bias-in-toxicity-classification\train.csv")

> Below files contain vocabulary and jigsaw all symbols

In [4]:
import pickle
  
# Open the file in binary mode
with open('preprocessing_data_final_pipeline.pkl', 'rb') as file:
    # Call load method to deserialze
    embeddings_index,jigsaw_symbols= pickle.load(file)


### 1. <i> Converting emojis to meaningful word <i>

In [5]:
# https://stackoverflow.com/questions/57580288/how-to-replace-emoji-to-word-in-a-text
import emoji
def emojis_to_word(text):
    text = emoji.demojize(text, delimiters=("", ""))
    text = text.replace('_',' ')
    return text
    

### 2. <i> Converting to Lower case <i>

In [6]:
# Data['comment_text'] = Data['comment_text'].str.lower()

### 3. <i>Decontraction of word<i>

In [7]:
#https://www.geeksforgeeks.org/nlp-expand-contractions-in-text-processing/
import contractions
def decontraction(text):
    expanded_words = []    
    for word in text.split():
      # using contractions.fix to expand the shotened words
        expanded_words.append(contractions.fix(word))   

    expanded_text = ' '.join(expanded_words)
    return expanded_text

### 4A. <i>Applying Lemmatization<i>

In [9]:
from nltk.stem import WordNetLemmatizer
def Lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    text = lemmatizer.lemmatize(str(text))
    return text

### 4B. <i>Applying Stemming <i>

In [12]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

### 5A. <i>Removing non english word <i>

> We will use custom vocab. glove vocab to drop non english word.

In [14]:
# ## Glove file
# from numpy import asarray
# # load the whole embedding into memory
# embeddings_index = dict()
# f = open(r"C:\Users\Lenovo\Downloads\glove.6B\glove.6B.100d.txt",encoding="utf8")
# for line in f:
#     values = line.split()
#     word = values[0]
#     coefs = asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()
# print('Loaded %s word vectors.' % len(embeddings_index))

In [15]:
def Non_english_word_dropping(text):
    final = []
    text_temp = text.split()
    for i in text_temp:
        if i in embeddings_index:
            final.append(i)
    text = ' '.join(final)
    return text

### 5B. <i>Removing non english Character/symbols and stop words <i>

In [22]:
from nltk.corpus import stopwords
# ", ".join(stopwords.words('english'))

In [23]:
STOPWORDS = set(stopwords.words('english'))
print(len(STOPWORDS))
STOPWORDS = list(STOPWORDS - {'no','nor','not'})
print(len(STOPWORDS))

179
176


In [24]:
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])


In [27]:
def remove_jigsaw_symbols(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', jigsaw_symbols))

In [29]:
''' lets write both in one Function'''
def remove_jigsaw_symbol_stop_word(text):
    ## Dont change order of calling function 1.remove_jigsaw_symbols and 2.remove_stopwords
    text = remove_jigsaw_symbols(text)
    text = remove_stopwords(text)
    return text

### 6. <i>Removing http/https links <i>

In [32]:
def removing_links(text):
    text = re.sub("(http|https)://[\w\-]+(\.[\w\-]+)+\S*", "", text)
    return text

### 6. <i>removal of Numbers,tabs and newline (\t,\n) <i>

In [34]:
def remove_num_tab_newline(text):
    text = re.sub(r"[\d\t\n]*", "", text)
    return text

### 7. <i>removal Punctuations(\t,\n) <i>

> we have already removed punctuation,number, when we removed jigsaw symbols. 

### 8. <i>removal extra space<i>

In [37]:
def remove_extra_speces(text):
    text = re.sub(r"[\s\s]+", " ", text)
    return text

In [40]:
# import pickle
# with open('preprocessing_data_final_pipeline.pkl', 'wb') as file:
#     pickle.dump([embeddings_index,jigsaw_symbols], file)

> Writing single function for preprocessing the data.

In [41]:
def preprocessing_DL(Data):
    ## Note : dont change preprocessing sequence
    ''' Converting Emojis to word'''
    Data["comment_text_pre"] = Data["comment_text"].apply(lambda text: emojis_to_word(text))
    '''Converting in Lower case'''
    Data["comment_text_pre"] = Data['comment_text_pre'].str.lower()
    '''Decontraction'''
    Data["comment_text_pre"] = Data["comment_text_pre"].apply(lambda text: decontraction(text))
    '''Lemmatization'''
    Data["comment_text_pre"] = Data["comment_text_pre"].apply(lambda text: Lemmatization(text))
    '''removing_links'''
    Data["comment_text_pre"] = Data["comment_text_pre"].apply(lambda text: removing_links(text))
    '''remove_jigsaw_symbol/character_stop_word'''
    Data["comment_text_pre"] = Data["comment_text_pre"].apply(lambda text: remove_jigsaw_symbol_stop_word(text))
    '''Non_english_word_dropping'''
    Data["comment_text_pre"] = Data["comment_text_pre"].apply(lambda text: Non_english_word_dropping(text))
    '''remove_num_tab_newline'''
    Data["comment_text_pre"] = Data["comment_text_pre"].apply(lambda text: remove_num_tab_newline(text))
    '''remove_extra_speces'''
    Data["comment_text_pre"] = Data["comment_text_pre"].apply(lambda text: remove_extra_speces(text))
    
    return Data["comment_text_pre"]

> Loding train tokenizer

In [42]:
import pickle
with open('train_tokenizer.pkl', 'rb') as file:
    tokenizer,max_length = pickle.load(file)

> Loding best model: Bidirectional_LSTM_128_128
* weighted AUC : 0.8946

In [276]:
from keras.models import load_model
filepath =r"C:\Users\Lenovo\Downloads\Self case study 2\Untitled Folder 1\my_best_model_2.hdf5"
model = load_model(filepath)

> ##  Final Function 
* Input: data frame (single row)
* Output : probability score  and class 

In [303]:
def Final_1(text):
#     data preprocessing
    text = preprocessing_DL(text) # Inpu: pass text as single row pandas data frame Output: list[single value]
    text.dropna(inplace=True)
    test_sequences = tokenizer.texts_to_sequences(text)
    test_padding = pad_sequences(test_sequences,padding='pre',maxlen=max_length)
    prediction =  model.predict(test_padding) 
    if prediction > 0.6:
        class_ = 'Toxic'
    else:
        class_ = 'Non Toxic'
    return prediction,class_

> ## Input :

In [309]:
d = {'comment_text': [" 'This is so cool. Its like, would you want your mother to read this?? Really great idea, well done!',"]}

> ## Output :

In [ ]:
d = {'comment_text': [" 'This is so cool. Its like, would you want your mother to read this?? Really great idea, well done!',"]}
text = pd.DataFrame(d)
predict_,class_ = Final_1(text)

In [310]:
text = pd.DataFrame(d)
predict_,class_ = Final_1(text)
print("Original Class: Non toixc",'\npredicted Class: ',class_,'\ntoxic score :',predict_[0][0])

Original Class: Non toixc 
predicted Class:  Non Toxic 
toxic score : 0.003443027


> ## Input :

In [311]:
d = {'comment_text':['hahahahahahahahhha suck it']}

> ## Output :

In [312]:
text = pd.DataFrame(d)
predict_,class_ = Final_1(text)
print("Original Class: toixc",'\npredicted Class: ',class_,'\ntoxic score :',predict_[0][0])

Original Class: toixc 
predicted Class:  Toxic 
toxic score : 0.9230589


> below data frame contain all text which contain identity group, we are checking how our modle is performing on these data set  

> ## Input :

In [300]:
d = {'comment_text': ['i am man',
                      'i am woman',
                      'i am lesbian', 
                      'i am gay man',
                      'i am dyke',
                      'i am white man',
                      'i am gay white man',
                      'i am black man',
                      'i am gay white man',
                      'i am black man',
                      'i am an gay white women',
                      'i am gay black man',
                      'i am black women',
                      'i am gay black women ',
                     'This is so cool. Its like, would you want your mother to read this?? Really great idea, well done!',
                     "This is such an urgent design problem; kudos to you for taking it on. Very impressive!",
                     "hahahahahahahahhha suck it.",
                     "It's ridiculous that these guys are being called protesters. Being armed is a threat of violence, which makes them terrorists.",
                     "Angry trolls, misogynists and Racists, oh my. It doesn't take all of my 150 IQ to see the slant here.  it's the Diversity diode at work yet again. We can say anything that we want because we are Diversity. You on the other hand must only  say what we allow you to say. From now on, winning arguments against any member of diversity will be considered offensive language.  facts, cogent, linear posts and Math are now verboten."]}
text = pd.DataFrame(d)
predict_ = Final_1(text)

> ## Output :

In [301]:
text['my score']= predict_
text['bias score'] =[0.20,0.41,0.51,0.57,0.60,0.66,0.66,0.77,0.78,0.80,0.80,0.82,0.85,0.87,0.0,0.0,0.45,0.5,0.6]
text

,comment_text,comment_text_pre,my score,bias score
0,i am man,man,0.023999,0.20
1,i am woman,woman,0.020597,0.41
2,i am lesbian,lesbian,0.308145,0.51
3,i am gay man,gay man,0.403672,0.57
4,i am dyke,dyke,0.128925,0.60
5,i am white man,white man,0.092565,0.66
6,i am gay white man,gay white man,0.512516,0.66
7,i am black man,black man,0.159100,0.77
8,i am gay white man,gay white man,0.512516,0.78
9,i am black man,black man,0.159100,0.80


## Observation:
* dropping data not good idea, using different data set might helpful
* dont make model more complex.
* maxpooling , average pooling is very helpful  when we want capture info from whols sequence rather, dont use last hidden state, bcoz this is classification task. not sequene translation.
* after experimentation i found that , bert will help us to caputure whole sequence.
    * EX. sequence contain any word except man or women  after or before the subgroup will give different result, so while creating embedding it always  good to use that techinique which will also preserve positionl information. (embedding layer or bidirectional LSTM or Bert)
* it dosent make sense to pass bert emebdding[poold output] to LSTM layer, bcoz ,lstm we use when we pass sequence , in this case sequence is  contain only embeddings of 1 word which is [start] 
* concatinnating differt layers will improve results.
* we have train more than 1 model . and we choose best model based on point estimate . which may be wrong ,average model may handle some cases where best model fail. always use CI  to choose base mdoel.